In [1]:
import chromadb
from chromadb.config import Settings


In [2]:
client = chromadb.Client()

In [3]:
collection = client.create_collection("my_collection")

In [4]:
pip install pandas


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
df = pd.read_csv("embedded_emails_all_miniLM.csv")

In [6]:
df

,id,chunks,chunks_embedded
0,<6501090.1075843349674.JavaMail.evans@thyme>,FYI. Commentary calls George W. Bush task gett...,"[-0.046354301273822784, -0.022795911878347397,..."
1,<6501090.1075843349674.JavaMail.evans@thyme>,"inauguration, President Bush's electricity pol...","[0.018535984680056572, -0.0205399040132761, 0...."
2,<6501090.1075843349674.JavaMail.evans@thyme>,Energy Regulatory Commission (FERC) signals FE...,"[0.07399576902389526, 0.06288360804319382, 0.0..."
3,<6501090.1075843349674.JavaMail.evans@thyme>,California. Skyrocketing retail electric price...,"[0.03702124208211899, -0.002088540932163596, 0..."
4,<6501090.1075843349674.JavaMail.evans@thyme>,fuses bombs lit passage Energy Policy Act (EPA...,"[-0.04222314432263374, 0.07322569191455841, 0...."
...,...,...,...
5765,<22527048.1075843384871.JavaMail.evans@thyme>,(E-mail)'; 'Cody Carter (E-mail)'; 'Curt Hatto...,"[-0.07114706188440323, 0.0022635660134255886, ..."
5766,<22527048.1075843384871.JavaMail.evans@thyme>,Lednicky (E-mail)'; 'Marty McFadden (E-mail)';...,"[-0.030311042442917824, -0.06628071516752243, ..."
5767,<22527048.1075843384871.JavaMail.evans@thyme>,committee soon possible meet discuss direction...,"[-0.029136644676327705, -0.06298520416021347, ..."
5768,<22527048.1075843384871.JavaMail.evans@thyme>,also set agenda Thursday person meeting. Thurs...,"[-0.05355614051222801, -0.02812141552567482, -..."


In [7]:
document = []
metaData = []

for i in range(len(df)) :
    document.append(df.iloc[i]["chunks"])
    metaData.append({"id" : df.iloc[i]["id"]})

In [8]:
import ast

In [9]:
batch_size = 500

# Parcourir le dataframe par lots
for batch_start in range(0, len(df), batch_size):
    # Créer un batch de données
    batch = df.iloc[batch_start:batch_start + batch_size]
    
    # Convertir les embeddings sous forme de chaîne en liste (utilisation de eval pour convertir de string à list)
    embeddings = batch['chunks_embedded'].apply(ast.literal_eval).tolist()

    # Extraire les documents du batch
    documents = batch['chunks'].tolist()  # Liste des chunks

    # Convertir les metadatas en dictionnaires (avec l'ID du mail comme clé)
    metadatas = [{'id': str(mail_id)} for mail_id in batch['id'].tolist()]

    # Créer des ids uniques pour chaque chunk
    chunk_ids = [f"{batch_start + idx}" for idx in range(len(batch))]  # Créer un id unique pour chaque chunk

    # Ajouter les documents et leurs métadonnées à la collection
    collection.add(
        metadatas=metadatas,  # Métadonnées sous forme de dictionnaire
        ids=chunk_ids,  # ID uniques pour chaque chunk (numérotés)
        documents=documents,  # Les chunks du document
        embeddings=embeddings  # Embeddings associés
    )

In [10]:
# Compter le nombre de documents dans la collection
num_documents = collection.count()
print(f"Il y a {num_documents} documents dans la collection.")


Il y a 5770 documents dans la collection.


In [11]:
result = collection.query(
    query_embeddings=embeddings[0],  # Exemple d'embedding pour la recherche
    n_results=5  # Limiter le nombre de résultats à 5
)

# Afficher les résultats
print(result)

{'ids': [['5500', '4652', '4651', '3031', '5499']], 'embeddings': None, 'documents': [['Steve= n=20 Schuman (schuman@haas.berkeley.edu) Thursday (March 15): Evening - Haas Technology Club Firm Night - Airport Marriot Hotel,=20 Burlingame, CA Friday (March 16): 3:30-4:45:? John Williams, SVP Marketing Intelligence Alliances, VI= SA=20 International - Wells Fargo Room - John Williams, Senior Vice President Market Intelligence Alliance= s=20 VISA International, speak experiences original member = of=20 Palm team, co-founder Razorfish graduate Haas an= d=20 Boalt schools. 5:00-??:?', "achievements already familiar many you. We're fortunate gentlemen aboard.? information available at: http://www.haas-abc.org/details/keynote.asp ? addition, outstanding lineup panelists, spread?across 6 panels: Biotech, E-Commerce, Entrepreneurship, Technology/Manufacturing, Telecom & Wireless, Venture Capital. firms represented: NTT, Sun, Sony, eSamsung, Sina.com, TSMC, Ericsson, SSMB, Walden International, 

In [13]:
# Récupérer un échantillon de documents depuis la collection
results = collection.query(
    query_embeddings=[0] * len(embeddings[0]),  # Utilisation de vecteurs nuls pour récupérer des documents
    n_results=1  # Limiter à 1 résultat pour récupérer la première ligne
)

# Afficher les résultats, en particulier le premier document
print("Premier document:")
print(f"IDs: {results['ids']}")
print(f"Documents: {results['documents']}")
print(f"Metadatas: {results['metadatas']}")



Premier document:
IDs: [['517']]
Documents: [["remedies Ameritech performance failures WISCONSIN -- Bipartisan bill restrict telemarketers introduced NORTH CAROLINA -- Commission orders NANPA reclaim 'NXX' codes OREGON -- PUC selects NeuStar interim number pooling administrator TENNESSEE -- Bill would adopt Lifeline/Link-Up Services Act OREGON -- PUC resolve UNE mapping issues HAWAII -- Counties would hold hearing wireless facility construction bill RHODE ISLAND -- Legislation would update telemarketing law OREGON -- Number complaints long"]]
Metadatas: [[{'id': '<25878449.1075843352551.JavaMail.evans@thyme>'}]]


In [14]:
print(df['chunks_embedded'].iloc[0])  # Affiche les embeddings du premier chunk


[-0.046354301273822784, -0.022795911878347397, 0.06693904846906662, 0.014837532304227352, 0.05049677938222885, 0.01832733489573002, -0.053233444690704346, 0.003616438712924719, -0.06215368211269379, -0.03772963210940361, -0.0645771250128746, 0.03820159286260605, -0.021201657131314278, -0.04062708839774132, 0.05253129079937935, -0.050735849887132645, 0.0234296265989542, -0.0466858446598053, 0.003993561957031488, -0.004270418547093868, 0.09445533156394958, -0.032903339713811874, 0.058278732001781464, -0.06387335062026978, 0.05314430221915245, 0.06506576389074326, -0.012689601629972458, 0.037282198667526245, -0.013506348244845867, -0.0358133502304554, -0.04414188861846924, -0.0479794517159462, -0.028094831854104996, 0.007869802415370941, 0.06819146871566772, 0.02253865823149681, -0.04079688340425491, 0.0472920760512352, 0.03312340006232262, 0.05169094353914261, 0.008569756522774696, -0.13954280316829681, 0.009540442377328873, -0.018456684425473213, -0.021161772310733795, -0.00236634723842

In [15]:
collection.get(include=['embeddings']).keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [16]:
dc = collection.get(include=['embeddings', "documents", "metadatas"])

In [17]:
for k in ['ids', 'uris', 'data', 'included'] :
    del dc[k]


In [18]:
dc.keys()

dict_keys(['embeddings', 'documents', 'metadatas'])

In [19]:
dc["embeddings"] = list(dc["embeddings"])

In [20]:
type(dc["metadatas"][0])

dict

In [21]:
dc["metadatas"][0]

{'id': '<6501090.1075843349674.JavaMail.evans@thyme>'}

In [22]:
dc["metadatas"] = list(map(lambda x : x["id"], dc["metadatas"]))

In [23]:
len(dc["documents"])


5770

In [24]:
len(dc["metadatas"])

5770

In [25]:
res = pd.DataFrame.from_dict(dc)

In [26]:
res

,embeddings,documents,metadatas
0,"[-0.046354301273822784, -0.022795911878347397,...",FYI. Commentary calls George W. Bush task gett...,<6501090.1075843349674.JavaMail.evans@thyme>
1,"[0.018535984680056572, -0.0205399040132761, 0....","inauguration, President Bush's electricity pol...",<6501090.1075843349674.JavaMail.evans@thyme>
2,"[0.07399576902389526, 0.06288360804319382, 0.0...",Energy Regulatory Commission (FERC) signals FE...,<6501090.1075843349674.JavaMail.evans@thyme>
3,"[0.03702124208211899, -0.002088540932163596, 0...",California. Skyrocketing retail electric price...,<6501090.1075843349674.JavaMail.evans@thyme>
4,"[-0.04222314432263374, 0.07322569191455841, 0....",fuses bombs lit passage Energy Policy Act (EPA...,<6501090.1075843349674.JavaMail.evans@thyme>
...,...,...,...
5765,"[-0.07114706188440323, 0.0022635660134255886, ...",(E-mail)'; 'Cody Carter (E-mail)'; 'Curt Hatto...,<22527048.1075843384871.JavaMail.evans@thyme>
5766,"[-0.030311042442917824, -0.06628071516752243, ...",Lednicky (E-mail)'; 'Marty McFadden (E-mail)';...,<22527048.1075843384871.JavaMail.evans@thyme>
5767,"[-0.029136644676327705, -0.06298520416021347, ...",committee soon possible meet discuss direction...,<22527048.1075843384871.JavaMail.evans@thyme>
5768,"[-0.05355614051222801, -0.02812141552567482, -...",also set agenda Thursday person meeting. Thurs...,<22527048.1075843384871.JavaMail.evans@thyme>


In [26]:
res.to_csv("GIST-small-Embedding-v0.csv",  index=False)

In [27]:
for i in range(100) :
    print('//\n')
    print(res.iloc[i]["documents"])
    print('//\n')

//

FYI. Commentary calls George W. Bush task getting competition electricity markets. Jim ----- Forwarded James Steffes/NA/Enron 02/06/2001 08:06 ----- "PennFuture" <pennfuture@pennfuture.org> 02/05/2001 03:33 PM Please respond "PennFuture" To: <Undisclosed-Recipient:@mailman.enron.com;> cc: Subject: PennFuture's E-cubed - Breaking Box PennFuture's E-cubed commentary biweekly email publication concerning current themes trends energy market. February 5, 2001 Vol. 3, No. 3 Breaking Box Two weeks
//

//

inauguration, President Bush's electricity policy already emerging crucible California characterized obedience free market ideology federalism confines federal government limited, backseat role. told California's crisis isolated event, result California's unique mistakes. view, federal government neither contributed crisis major role play solving it. Wholesale market caps level declared inappropriate intrusion free markets make things worse. Moreover, appointment Curt Hebert new Chairman

In [28]:
print(res.iloc[1]["documents"])

inauguration, President Bush's electricity policy already emerging crucible California characterized obedience free market ideology federalism confines federal government limited, backseat role. told California's crisis isolated event, result California's unique mistakes. view, federal government neither contributed crisis major role play solving it. Wholesale market caps level declared inappropriate intrusion free markets make things worse. Moreover, appointment Curt Hebert new Chairman Federal


In [29]:
print(res.iloc[147]["documents"])

proposal? Jim


In [30]:
pip install torch transformers tqdm


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from tqdm import tqdm

# Charger le modèle et le tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Fonction de pooling pour obtenir l'embedding de la phrase
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # Embeddings des tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # Moyenne des embeddings des tokens en prenant en compte le masque d'attention
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Fonction pour calculer les embeddings
def compute_embeddings_batch(batch_texts):
    encoded_input = tokenizer(batch_texts, padding=True, truncation=True, max_length=500, return_tensors='pt').to(model.device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # Normalisation de l'embedding
    return F.normalize(sentence_embeddings, p=2, dim=1).cpu().tolist()

# Fonction pour traiter les données en lots et calculer les embeddings
def process_in_batches(df, batch_size=500):
    all_embeddings = []
    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df['chunks'][i:i+batch_size].tolist()  # Extraire les textes (chunks) pour le batch
        batch_embeddings = compute_embeddings_batch(batch_texts)
        all_embeddings.extend(batch_embeddings)  # Ajouter les embeddings calculés
    return all_embeddings

# Exemple d'utilisation pour une seule question
question = ["What was the impact of President Bush's electricity policy on California's crisis?"]

# Calculer l'embedding pour la question donnée
question_embedding = compute_embeddings_batch(question)

# Afficher l'embedding de la question
print(question_embedding)


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[-0.006061376538127661, 0.057496026158332825, 0.08485960960388184, 0.04815136641263962, 0.031404927372932434, 0.033542901277542114, -0.00026581247220747173, 0.035249702632427216, -0.07964485138654709, 0.0012700402876362205, 0.0560288168489933, 0.042307768017053604, 0.057439450174570084, -0.021935807541012764, 0.008544374257326126, 0.022930767387151718, -0.039935991168022156, -0.0787452980875969, -0.054526880383491516, 0.0008081780979409814, 0.08421900123357773, 0.06079871952533722, -0.021885987371206284, -0.0325327143073082, 0.04196997731924057, 0.06900451332330704, -0.0067999097518622875, 0.061380576342344284, -0.08010152727365494, -0.006282326299697161, -0.0025818049907684326, -0.06948334723711014, -0.04328533634543419, -0.060932401567697525, 0.03512980416417122, 0.044876065105199814, 0.0157150961458683, 0.01031945738941431, -0.026645757257938385, 0.020915798842906952, -0.010417114943265915, -0.03963593393564224, 0.0033625736832618713, 0.018160497769713402, -0.05363071709871292, 0.0

In [32]:
# Effectuer la requête en utilisant l'embedding de la question
result = collection.query(
    query_embeddings=question_embedding,  # L'embedding de la question
    n_results=5  # Limiter le nombre de résultats à 5
)

# Afficher les résultats
print(result)


{'ids': [['1', '2731', '3591', '2922', '852']], 'embeddings': None, 'documents': [["inauguration, President Bush's electricity policy already emerging crucible California characterized obedience free market ideology federalism confines federal government limited, backseat role. told California's crisis isolated event, result California's unique mistakes. view, federal government neither contributed crisis major role play solving it. Wholesale market caps level declared inappropriate intrusion free markets make things worse. Moreover, appointment Curt Hebert new Chairman Federal", "Power Politics: Energy Crisis Offers Clues Workings Bush=20 Administration --- Cheney Heavy Lifting Bush Rides Agenda; Taste f= or=20 Horse-Trading --- `A Lot Pain West' Wall Street Journal, 02/16/2001 Generators: Plans Build Peaking-Power Plants Calif Dow Jones, 02/16/2001 Developments California's Electricity Crisis Associated Press Newswires, 02/16/2001 Bush Pushes Energy Plan Mexico Trade: President Wants

In [36]:
for i in range (5):
    print ('Mail ID:', result['ids'][0][i])
    print ('Document:', result['documents'][0][i])


Mail ID: 1
Document: inauguration, President Bush's electricity policy already emerging crucible California characterized obedience free market ideology federalism confines federal government limited, backseat role. told California's crisis isolated event, result California's unique mistakes. view, federal government neither contributed crisis major role play solving it. Wholesale market caps level declared inappropriate intrusion free markets make things worse. Moreover, appointment Curt Hebert new Chairman Federal
Mail ID: 2731
Document: Power Politics: Energy Crisis Offers Clues Workings Bush=20 Administration --- Cheney Heavy Lifting Bush Rides Agenda; Taste f= or=20 Horse-Trading --- `A Lot Pain West' Wall Street Journal, 02/16/2001 Generators: Plans Build Peaking-Power Plants Calif Dow Jones, 02/16/2001 Developments California's Electricity Crisis Associated Press Newswires, 02/16/2001 Bush Pushes Energy Plan Mexico Trade: President Wants Cut Hemispheric= =20 Barriers Oil, Gas El